# 小地域の収入推定

```Data```フォルダには、小地域の収入データ(```income_district.csv```)と国勢調査の小地域集計データが格納されています。

- 国勢調査の小地域集計データでその地域の収入を推定するための機械学習・深層学習モデルを構築しなさい。
- モデルの精度を評価し、出来るだけ精度が高いのモデルを得るよう、適切な特徴量エンジニアリングやモデル選定の考えもまとめてください。

In [223]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [149]:
h27_age_df = pd.read_csv("Data/h27_age_df.csv")
h27_family_df = pd.read_csv("Data/h27_family_df.csv")
h27_gender_df2 = pd.read_csv("Data/h27_gender_df2.csv")
h27_house_df = pd.read_csv("Data/h27_house_df.csv")
h27_house_info_df = pd.read_csv("Data/h27_house_info_df.csv")
h27_indusry_df = pd.read_csv("Data/h27_indusry_df.csv")
h27_job_df = pd.read_csv("Data/h27_job_df.csv")
h27_labor_df = pd.read_csv("Data/h27_labor_df.csv")
h27_marriage_df = pd.read_csv("Data/h27_marriage_df.csv")
h27_work_status_df = pd.read_csv("Data/h27_work_status_df.csv")
income_df=pd.read_csv("Data/income_df.csv")

In [219]:
income_df
income_df.rename(columns={'area_code': 'district_id'}, inplace=True)
income_df

,district_id,income_mean
0,13228,480.984419
1,43514,439.775281
2,23237,487.507769
3,18208,505.495103
4,12238,385.423846
...,...,...
1240,14137,601.266088
1241,2204,379.073909
1242,16207,531.328234
1243,13107,512.674825


In [240]:
age_df = h27_age_df[h27_age_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
age= age_df.drop(columns_to_delete, axis=1)
age=age.drop(age.columns[20:64], axis=1)
age

,district_id,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100
0,45201,19220,20777,17510,18611,22131,26144,29081,25070,24686,25425,28638,30260,21673,17872,15095,9741,4347,1250,237
620,45202,7897,7689,6393,7661,9132,9783,10200,8775,9707,11366,12789,12870,9228,8943,8083,5453,2274,642,133
886,45203,5950,5980,4499,5139,6176,7096,7846,6846,7538,8356,9900,10505,7678,7350,6713,4269,1805,503,81
1265,45204,2187,2294,1807,1957,2548,2813,2904,2634,3242,4012,4555,4702,3760,3614,3352,2193,962,265,36
1548,45205,2056,1962,1279,1788,2315,2515,2475,2278,2722,3373,3912,3796,2930,2872,2744,1918,933,267,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251086,8447,347,437,381,383,347,428,549,592,652,692,771,847,629,653,505,354,121,33,4
251112,8521,1095,1071,1062,1197,1232,1425,1401,1249,1267,1586,1843,1761,1161,991,842,689,311,62,9
251162,8542,358,389,393,469,416,515,557,545,574,673,874,823,514,428,272,241,102,22,5
251181,8546,1227,1175,1089,1111,1398,1543,1837,1554,1501,1660,1964,2027,1385,1120,937,605,299,75,5


In [242]:
family_df=h27_family_df[h27_family_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
family=family_df.drop(columns_to_delete, axis=1)
family=family.drop(family.columns[3:11], axis=1)
family


,district_id,family_count,family_member_count
0,25201,136153,340973
593,25202,45546,113679
998,25203,41788,118193
1320,25204,29784,81312
1512,25206,60224,137247
...,...,...,...
250929,5366,1563,4986
250996,5368,796,3110
251024,5434,6152,20279
251195,5463,4807,15319


In [244]:
gender_df=h27_gender_df2[h27_gender_df2['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name','family_count']
gender= gender_df.drop(columns_to_delete, axis=1)
gender


,district_id,population,male_population,female_population
0,18201,265904,128892,137012
828,18202,66165,32558,33607
994,18204,29670,14539,15131
1145,18205,33109,15683,17426
1364,18206,24125,11478,12647
...,...,...,...,...
251010,40621,34963,17646,17317
251078,40625,20243,9382,10861
251170,40642,6627,3077,3550
251181,40646,7458,3509,3949


In [246]:
house_df=h27_house_df[h27_house_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
house= house_df.drop(columns_to_delete, axis=1)
house


,district_id,family_house,own_house_family,public_rent_family,private_rent_family,salary_rent_family,room_rent_family,other_house_family,own_house_individual,public_rent_individual,private_rent_individual,salary_rent_individual,room_rent_individual,other_house_individual
0,25201,135957,97948,5183,27685,2501,777,1856,269022,9676,46261,5570,1585,2746
593,25202,45473,29873,808,11932,1487,246,1127,86659,1641,18925,2597,499,1472
998,25203,41730,31174,929,7372,860,234,1161,98331,2024,12801,1449,504,1425
1320,25204,29751,22222,1139,4974,699,162,555,66356,2351,9204,1247,319,801
1512,25206,60180,34073,842,21372,2001,364,1528,98803,1836,29116,3550,698,1843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250994,22342,16404,9224,225,5596,902,107,350,26792,514,11349,1947,237,416
251050,22344,6378,4555,396,692,546,25,164,13960,868,1399,1236,57,225
251133,22424,10239,7152,153,2389,192,60,293,22977,368,4607,365,111,307
251139,22429,2878,2477,96,87,86,13,119,6482,208,167,129,18,122


In [263]:
house_info_df=h27_house_info_df[h27_house_info_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
house_info= house_info_df.drop(columns_to_delete, axis=1)
house_info.replace('-', 0, inplace=True)
house_info

,district_id,single_house_family,long_house_family,share_house_family,1-2floor_family,3-5floor_family,6-10floor_family,11floor_family,single_house_individual,long_house_individual,share_house_individual,1-2floor_individual,3-5floor_individual,6-10floor_individual,11floor_individual
0,27102,9431,954,40447,1045,9030,13457,16915,23379,1773,76699,1400,14645,21284,39370
62,27103,7531,1157,28069,463,3839,8476,15291,19168,2306,49349,608,5880,13158,29703
111,27104,9049,424,20069,624,4384,4073,10988,22738,775,40944,884,7699,7484,24877
190,27106,4810,194,46117,106,2276,14654,29081,11999,377,77749,140,3280,21200,53129
268,27107,11038,488,27651,367,7847,8930,10507,27296,867,51004,471,12498,14357,23678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251219,19425,1502,13,204,49,66,89,0,4581,24,312,75,109,128,0
251224,19429,961,1,29,15,14,0,0,2741,1,59,31,28,0,0
251227,19430,6927,116,2286,1190,900,196,0,20518,247,3897,2137,1456,304,0
251240,19442,303,6,22,10,12,0,0,676,20,24,10,14,0,0


In [206]:
indusry_df=h27_indusry_df[h27_indusry_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
industry=indusry_df.drop(columns_to_delete, axis=1)
industry

,district_id,A_agriculture_forestry_male,agriculture_male,B_fishing_male,C_mining_male,D_construction_male,E_manufacturing_male,F_Electricity_male,G_information_male,H_transport_male,...,K_real_estate_female,L_academic_female,M_accommodation_female,N_entertainment_female,O_education_female,P_health_care_female,Q_service_female,R_other_service_female,S_public_services_female,T_other_female
0,43101,306,264,6,2,3296,3683,419,1636,1773,...,1127,1335,3898,2150,3058,9432,275,2841,1391,1989
248,43102,776,749,14,4,6089,5470,282,1260,2813,...,919,1178,2961,2123,2624,10802,272,2398,1262,1673
493,43103,1695,1608,230,2,2339,2099,140,588,1773,...,347,438,1637,891,1107,4665,149,1328,539,1025
657,43104,1830,1808,203,3,3952,3567,194,696,2395,...,471,573,1805,1276,1480,7494,223,1519,577,1328
894,43105,2169,2136,4,6,4067,5572,205,742,2078,...,490,760,2194,1418,2101,7398,270,1645,687,1313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251158,24471,220,154,124,-,299,460,10,13,127,...,16,8,119,61,101,505,47,76,38,5
251176,24472,216,190,621,28,430,304,8,5,166,...,10,37,202,118,97,559,99,93,61,31
251215,24543,175,97,404,5,606,649,32,24,249,...,14,25,284,89,186,906,40,92,84,35
251238,24561,516,503,27,11,314,175,7,11,104,...,17,21,150,50,99,561,27,52,47,1


In [207]:
job_df=h27_job_df[h27_job_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
job=job_df.drop(columns_to_delete, axis=1)
job

,district_id,A_administrative_male,B_professional_workers_male,C_office_workers_male,D_sales_workers_male,E_service_workers_male,F_safety_workers_male,G_agriculture_workers_male,H_production_workers_male,I_transport_workers_male,...,C_office_workers_female,D_sales_workers_female,E_service_workers_female,F_safety_workers_female,G_agriculture_workers_female,H_production_workers_female,I_transport_workers_female,J_construction_workers_female,K_cleaning_workers_female,L_other_wokers_female
0,13101,1391,3359,2438,1697,765,793,3,402,88,...,3291,1127,1023,65,1,235,8,16,192,1823
144,13102,2833,8027,6731,6424,2400,573,10,1359,574,...,10997,3395,3501,90,2,803,46,27,762,4759
268,13103,4869,11124,7726,6303,2761,793,39,1490,658,...,12420,3833,4236,103,12,844,54,26,963,8841
415,13104,3984,16054,11658,9109,5533,2472,69,2932,1155,...,17201,6288,6929,263,19,1618,70,53,1987,12973
589,13105,3451,15083,9427,7308,3182,797,47,2431,701,...,15001,4595,4270,86,12,1331,30,52,1147,5469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251116,23447,277,1283,1860,602,581,258,181,4566,955,...,2319,1061,1672,22,116,1088,32,14,729,139
251126,23501,234,1534,1690,707,478,258,416,4153,500,...,1848,952,1386,10,326,1643,27,31,711,262
251167,23561,54,128,182,71,85,36,292,209,83,...,207,86,267,7,203,104,2,3,80,4
251203,23562,44,69,104,49,49,25,93,146,75,...,141,64,225,-,27,67,3,5,48,-


In [209]:
labor_df=h27_labor_df[h27_labor_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
labor=labor_df.drop(columns_to_delete, axis=1)
labor

,district_id,labor_male,no_labor_male,labor_female,no_labor_female
0,23101,45761,15806,33891,32964
429,23102,21403,6637,17626,14209
542,23103,44853,17606,34961,33698
945,23104,43786,14240,32761,27786
1172,23105,38918,14117,29065,24806
...,...,...,...,...,...
251081,12426,2173,1050,1508,1903
251131,12427,2384,1274,1654,2226
251165,12441,2645,1575,1939,2714
251229,12443,1740,1429,1425,2194


In [212]:
marriage_df=h27_marriage_df[h27_marriage_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
marriage=marriage_df.drop(columns_to_delete, axis=1)
marriage

,district_id,unmarried,married,divorced,unmarried_male,married_male,divorced_male,unmarried_female,married_female,divorced_female
0,39201,76365,153847,47056,38837,76315,11447,37528,77532,35609
479,39202,2542,6899,2944,1550,3490,784,992,3409,2160
488,39203,3783,8442,3302,2194,4245,851,1589,4197,2451
545,39204,11007,22844,7150,6073,11376,1699,4934,11468,5451
658,39205,6047,13398,4524,3638,6712,1123,2409,6686,3401
...,...,...,...,...,...,...,...,...,...,...
250040,1668,1603,4260,1437,888,2135,330,715,2125,1107
250183,1691,2799,8198,1899,1785,4156,530,1014,4042,1369
250237,1692,4554,12270,3229,2653,6189,883,1901,6081,2346
251125,1693,999,2800,762,606,1416,213,393,1384,549


In [213]:
work_status_df=h27_work_status_df[h27_work_status_df['district2_id'].isna()]
columns_to_delete = ['district2_id','level_identifier','state_name','city_name','district_name','district2_name']
work_status=work_status_df.drop(columns_to_delete, axis=1)
work_status

,district_id,employer_male,self_employed_male,family_work_male,unkonwn_status_male,employer_female,self_employed_female,family_work_female,unkonwn_status_female
0,37201,91038,11731,957,2525,75462,3823,4209,1684
278,37202,23937,3237,319,728,19577,1153,1264,568
432,37203,10940,1785,125,285,9371,496,657,169
549,37204,7378,1174,78,117,5535,364,506,87
599,37205,12266,2818,383,445,10871,731,1536,356
...,...,...,...,...,...,...,...,...,...
250983,11385,7027,1128,140,266,5844,236,474,183
251070,11408,7829,1303,137,302,6188,323,508,168
251094,11442,7903,941,87,350,5547,288,292,173
251159,11464,10542,1436,190,508,7761,368,483,318


In [264]:
merged_df = pd.merge(age, family, on="district_id")
merged_df = pd.merge(merged_df, gender, on="district_id")
merged_df = pd.merge(merged_df, house, on="district_id")
merged_df = pd.merge(merged_df, house_info, on="district_id")
merged_df = pd.merge(merged_df, industry, on="district_id")
merged_df = pd.merge(merged_df, job, on="district_id")
merged_df = pd.merge(merged_df, labor, on="district_id")
merged_df = pd.merge(merged_df, marriage, on="district_id")
merged_df = pd.merge(merged_df, work_status, on="district_id")
merged_df

,district_id,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,...,married_female,divorced_female,employer_male,self_employed_male,family_work_male,unkonwn_status_male,employer_female,self_employed_female,family_work_female,unkonwn_status_female
0,45201,19220,20777,17510,18611,22131,26144,29081,25070,24686,...,98959,38679,80219,13952,1541,2791,76534,3968,6097,2127
1,45202,7897,7689,6393,7661,9132,9783,10200,8775,9707,...,41355,19289,32431,7187,795,514,30615,1731,3380,379
2,45203,5950,5980,4499,5139,6176,7096,7846,6846,7538,...,30799,16045,24474,4670,429,475,22419,1491,1723,316
3,45204,2187,2294,1807,1957,2548,2813,2904,2634,3242,...,13740,7379,10444,2075,289,88,9339,610,1039,74
4,45205,2056,1962,1279,1788,2315,2515,2475,2278,2722,...,12020,6075,8073,3119,407,176,8309,725,1798,134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,8447,347,437,381,383,347,428,549,592,652,...,2397,1092,2039,552,66,14,1674,95,228,10
1892,8521,1095,1071,1062,1197,1232,1425,1401,1249,1267,...,5703,1927,5147,1300,393,144,3637,190,878,97
1893,8542,358,389,393,469,416,515,557,545,574,...,2303,724,2194,399,54,57,1694,65,136,36
1894,8546,1227,1175,1089,1111,1398,1543,1837,1554,1501,...,6363,2206,5619,1268,265,249,4451,222,797,143


In [265]:
final_df = pd.merge(merged_df, income_df, on="district_id")
final_df
cleaned_data = final_df.dropna(subset=['income_mean'])
cleaned_data

,district_id,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,...,divorced_female,employer_male,self_employed_male,family_work_male,unkonwn_status_male,employer_female,self_employed_female,family_work_female,unkonwn_status_female,income_mean
0,45201,19220,20777,17510,18611,22131,26144,29081,25070,24686,...,38679,80219,13952,1541,2791,76534,3968,6097,2127,410.041038
1,45202,7897,7689,6393,7661,9132,9783,10200,8775,9707,...,19289,32431,7187,795,514,30615,1731,3380,379,347.089094
2,45203,5950,5980,4499,5139,6176,7096,7846,6846,7538,...,16045,24474,4670,429,475,22419,1491,1723,316,363.127201
3,45204,2187,2294,1807,1957,2548,2813,2904,2634,3242,...,7379,10444,2075,289,88,9339,610,1039,74,361.800658
4,45205,2056,1962,1279,1788,2315,2515,2475,2278,2722,...,6075,8073,3119,407,176,8309,725,1798,134,345.428894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,8442,703,707,673,668,933,1004,1187,1094,1097,...,1545,4122,561,55,17,2729,126,194,17,470.108696
1234,8443,2183,2331,2528,2370,2732,3164,3783,3103,2693,...,3964,11388,1453,162,352,8812,424,501,205,445.899321
1235,8521,1095,1071,1062,1197,1232,1425,1401,1249,1267,...,1927,5147,1300,393,144,3637,190,878,97,526.232114
1236,8546,1227,1175,1089,1111,1398,1543,1837,1554,1501,...,2206,5619,1268,265,249,4451,222,797,143,478.836317


In [267]:
cleaned_data.replace('-', 0, inplace=True)
X = cleaned_data.drop("income_mean", axis=1)
y = cleaned_data["income_mean"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import r2_score
print('testing score:', r2_score(y_test, y_pred))

testing score: 0.45354413223815904
